In [1]:
#Predict smallest Number

#!conda install -n mldds -c anaconda joblib
import os
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings(action='once')

import multiprocessing
num_cores = multiprocessing.cpu_count()

print("Cores: ", num_cores)

import time
import keras
from sklearn.preprocessing import LabelEncoder, StandardScaler
# import tensorflow as tf
# config = tf.ConfigProto( device_count = {'GPU': 0 , 'CPU': num_cores} )
# sess = tf.Session(config=config) 
# keras.backend.set_session(sess)

#os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from MyTotoResearchv4 import *

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, Lasso, SGDRegressor, SGDClassifier, LogisticRegression
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.tree import DecisionTreeRegressor  
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor
from sklearn.svm import SVR



Cores:  6


/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
Using TensorFlow backend.
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/opt/conda/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  return _inspect.getargspec(target)
/opt/conda/lib/python3.6/site-packages/tensorflow/python/keras/backend.py:4900: ResourceWarning: unclosed file <_io.TextIOWrapper name='/home/jovyan/.keras/keras.json' mode='r' encoding='UTF-8'>
  _config = json.load(open(_config_path))
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __nam

Done.


In [2]:
def getAllData(df):
    drop_cols = ['T', 'D', 'N1','N2','N3','N4','N5','N6','N7','L','M','S','R','E','A','V' ,'J','U']
    drop_cols = ['D', 'N1','N2','N3','N4','N5','N6','N7', 'Ph', 'il', 'age', 'dist', 'adia', 'sundist', 'sunadia' ]

#     Ph         1521 non-null float64
# il         1521 non-null float64
# age        1521 non-null float64
# dist       1521 non-null float64
# adia       1521 non-null float64
# sundist    1521 non-null float64
# sunadia    1521 non-null float64

#    drop_cols = ['T', 'D', 'M','S','R','E','A','V' ,'J','U']


    X = df.drop(drop_cols, axis=1)
    
    X = df[['T','L','M','S','R','E','A','V']]
    X = df[[ 'T','L','M','S','il', 'age', 'dist', 'adia', 'sundist', 'sunadia']]

#     df1 = df[['N1','N2','N3','N4','N5','N6','N7']]
#     X['smallest'] = df1.min(axis=1)
#     X['biggest'] = df1.max(axis=1)

    return X


def getSmallestN(df):
    df1 = df[['N1','N2','N3','N4','N5','N6','N7']]
    y = pd.DataFrame(index=df.index)
    y['SN'] = df1.min(axis=1)
    return y ;

def getBiggestN(df):
    df1 = df[['N1','N2','N3','N4','N5','N6','N7']]
    y = pd.DataFrame(index=df.index)
    y['LN'] = df1.max(axis=1)
    return y ;



In [3]:
seed = 42

mtr = MyTotoResearch(algo_no=1)
lresult, df = mtr.load_totodata()

X = mtr.modified_dataset(getAllData(df)) #

#X

#getSmallestN(df)
#getBiggestN(df)

Loaded MyTotoResearch algo_no:  1
1521
      N1  N2  N3  N4  N5  N6  N7         D
0      3  10  18  21  29  36  42  20040212
1      6   8  22  39  40  41  45  20040216
2      5   8  15  17  21  35  36  20040219
3      6  20  21  23  28  38  39  20040223
4      3  10  16  25  26  40  44  20040226
5     14  20  22  25  27  28  29  20040301
6      2   6   7  25  40  41  43  20040304
7      2  17  18  19  21  42  44  20040308
8     10  18  24  26  34  36  43  20040311
9      1   9  13  16  32  35  45  20040315
10     6  17  21  31  40  42  43  20040318
11    16  21  22  23  27  28  35  20040322
12     6  13  18  20  37  38  43  20040325
13     1   2   6   7  19  24  32  20040329
14     3   9  27  28  31  39  40  20040401
15     4   7  12  13  15  16  33  20040405
16     4  11  25  30  33  36  39  20040408
17     2   8  10  16  29  32  37  20040412
18     2   3   9  19  26  32  34  20040415
19    10  15  18  26  36  39  45  20040419
20     5   6   8  14  23  26  37  20040422
21     7  15  1

In [4]:
model_result = {}
def store_model_result(sEstimator, N, r):
    if ( sEstimator not in model_result ):
        model_result[sEstimator] = pd.DataFrame(index=df.index) 
    m_df = model_result[sEstimator] ;
    m_df[N] = r
    return m_df
    


In [16]:
dt_lr = LinearRegression(fit_intercept=True) # 22.65% intersection
#dt_dt = DecisionTreeRegressor(max_features='sqrt', splitter='random', min_samples_split=4, max_depth=3)
#dt = AdaBoostRegressor(random_state=42,base_estimator=dt_lr,learning_rate=1)

#regr_1 = DecisionTreeRegressor(max_depth=4)
dt = AdaBoostRegressor(DecisionTreeRegressor(max_depth=4),
                          n_estimators=300, random_state=42)
dt = SVR(kernel='linear',degree=1,gamma='scale')

bBest = 0.0
iIndex = 0
def analyzeData(y_c):
    global bBest
#    model = SVR(kernel='rbf',epsilon=1.0,gamma='scale') ;
    model = LinearRegression(fit_intercept=True) ;
    model.fit(X_scaled, y_c)
#        p = store_model_result(sEstimator, "N", model.predict(X_scaled).round())
    p = model.predict(X_scaled).round()
    m_df = pd.DataFrame(index=df.index) 
    m_df['N'] = p
    m_df['NU'] = np.ceil(p)
    m_df['ND'] = np.floor(p)


    p = m_df
    bAccuracy = mtr.getAccuracyCount(np.array(p)) ;
    if (iIndex % 2000 == 0 ):
        print(str(iIndex/2000), end = ' ')
    if ( bBest < bAccuracy or bAccuracy > 90.0 ):
        print ( str(iIndex) + " Accuracy: ",  bAccuracy)
        bBest = bAccuracy
#            mtr.plot_matched_counts(p.values)
#        mtr.print_predictions(result)



In [17]:
from itertools import product
data = [[1,2,3],[4,5,6],[7,8,9],[10,11,23]]
def getYLimited(oldCombination, row, col):
    global iIndex
    newCombination = oldCombination + str(data[row][col]) + ","
    if (row == len(data) - 1):
        analyzeData(newCombination[0:-1].split(","))
        iIndex = iIndex + 1
    if (row < len(data) - 1):
        getYLimited(newCombination, row + 1, 0);
    if (col < len(data[row]) - 1):
        getYLimited(oldCombination, row, col + 1);

# data_comb = []
# getYLimited_start_pos = 0
# getYLimited("",0, 1, 11);
# print(data_comb)
# getYLimited_start_pos = 0
# data_comb = []
# max_comb = 5
# getYLimited("",0, 1, 5);
# print(data_comb)
# getYLimited_start_pos = 5
# data_comb = []
# getYLimited("",0, 1, 5);
# print(data_comb)
# getYLimited_start_pos = 10
# data_comb = []
# getYLimited("",0, 1, 3);
# print(data_comb)

In [ ]:


# scaler = StandardScaler()
# scaler.fit(X.drop('T',axis=1))

# X_scaled = scaler.transform(X.drop('T',axis=1))
X_scaled = X

l_encoder = LabelEncoder()
result = pd.DataFrame(index=df.index)
result['T'] = df['T']

test_data = mtr.get_test_data()

y_combination = pd.DataFrame(index=df.index)
for i in range(1,8):
     y_combination['N'+str(i)] = df['N'+str(i)]
#y_combination.reset_index()
data = y_combination.values
check_size = len(data)
print("#of Entries: ", check_size)
print(data)

model_result = {}
bBest = 0.0
iIndex = 0
getYLimited("",0, 1);

#dt = LinearRegression(fit_intercept=False) # 22.65% intersection
#dt = DecisionTreeRegressor()
#max_features='sqrt', splitter='random', min_samples_split=4, max_depth=3) ;
#max_comb = 2000

# bBest = 0.0
# getYLimited_start_pos = 0
# for idx in range(1,0):
#     data_comb = []
#     getYLimited_start_pos = (idx-1) * max_comb
#     getYLimited(data_comb,"",0, 0, max_comb);
#     print(((idx-1) * max_comb), end=' ')
# #     if ( len(data_comb[0]) < check_size ):
# #         print("ERROR generating combination. ", len(data_comb[0]), " expected ", check_size)
# #         break 
#     for y_c in range(0,len(data_comb)):
#         sEstimator = "DT" + str(y_c)
#         #     model = Pipeline([('poly', PolynomialFeatures(degree=deg)),
#         #                       ('linear', AdaBoostRegressor(random_state=seed,base_estimator=dt,learning_rate=1))])
#         model = dt ;
#         model.fit(X_scaled, data_comb[y_c])
# #        p = store_model_result(sEstimator, "N", model.predict(X_scaled).round())
#         p = model.predict(X_scaled).round()
#         m_df = pd.DataFrame(index=df.index) 
#         m_df['N'] = p
#         p = m_df
    
# #    print(p)
# #     dt = DecisionTreeRegressor()
# #     dt = DecisionTreeRegressor(max_features='sqrt', splitter='random', min_samples_split=4, max_depth=3) ;
#         bAccuracy = mtr.getAccuracyCount(np.array(p)) ;
#         if ( bBest < bAccuracy or bAccuracy > 98.0 ):
#             print ( str(idx) + " " + sEstimator + " Accuracy: ",  bAccuracy)
#             bBest = bAccuracy
# #            mtr.plot_matched_counts(p.values)
#     #        mtr.print_predictions(result)

print("Done.")

#of Entries:  1521
[[  3.  10.  18. ...,  29.  36.  42.]
 [  6.   8.  22. ...,  40.  41.  45.]
 [  5.   8.  15. ...,  21.  35.  36.]
 ..., 
 [  8.  10.  16. ...,  30.  37.  44.]
 [  7.  13.  21. ...,  29.  35.  37.]
 [  5.  17.  24. ...,  45.  46.  49.]]
0 0 Accuracy:  13.20754716981132
1693 Accuracy:  15.09433962264151
2000 4000 6000 8000 9213 Accuracy:  16.9811320754717
10000 11992 Accuracy:  18.867924528301888
12000 14000 16000 18000 20000 22000 24000 26000 28000 30000 32000 34000 36000 38000 40000 42000 44000 46000 48000 50000 52000 54000 56000 58000 60000 62000 64000 66000 68000 70000 72000 74000 76000 78000 80000 82000 84000 86000 88000 90000 92000 94000 96000 98000 100000 102000 104000 106000 108000 110000 112000 114000 116000 118000 120000 122000 124000 126000 128000 130000 132000 134000 136000 138000 140000 142000 144000 146000 148000 150000 152000 154000 156000 158000 160000 162000 164000 166000 168000 170000 172000 174000 176000 178000 180000 182000 184000 186000 188000 1900

In [ ]:
def initialize():
    model_result = {}
    bBest = 0.0
    iIndex = 0

In [ ]:
already_printed = {}
bBest = 0.0
print ( "BEST INTERSECTION")
for key1, value1 in model_result.items():
    for key2, value2 in model_result.items():
        if ( key1 != key2 )
            if ( (key1 + key2) not in already_printed ):
                if ( (key2 + key1) not in already_printed ) :
                    already_printed[key1+key2] = 'Y'
                    p = mtr.getIntersection(value1.values, value2.values)
                    bAccuracy = mtr.getAccuracyCount(np.array(p)) ;
                    if ( bBest < bAccuracy ):
                        print ( key1, key2, " Accuracy: ",  bAccuracy)
                        bBest = bAccuracy
                        mtr.plot_matched_counts(p)
#                        mtr.print_predictionsV2(np.array(p))



In [ ]:
already_printed = {}
bBest = 0.0
print ( "BEST UNION")
for key1, value1 in model_result.items():
    for key2, value2 in model_result.items():
        if ( key1 != key2 ):
            if ( (key1 + key2) not in already_printed ):
                if ( (key2 + key1) not in already_printed ) :
                    already_printed[key1+key2] = 'Y'
                    p = [np.union1d(a,b) for a,b in zip(value1.values, value2.values)] #mtr.getIntersection(value1.values, value2.values)
                    bAccuracy = mtr.getAccuracyCount(np.array(p)) ;
                    if ( bBest < bAccuracy ):
                        print ( key1, key2, " Accuracy: ",  bAccuracy)
                        bBest = bAccuracy
                        mtr.plot_matched_counts(p)
mtr.print_predictionsV2(np.array(p))



In [ ]:
columns = result.columns #['N'+str(i)+'_P' for i in range(1,8)]
print(result.columns)
my_prediction = pd.DataFrame(result, columns=columns)

print(my_prediction)
for col in columns:
    my_prediction[col] = my_prediction[col].apply(lambda x: int(x) if x == x else "")
    
#print(my_prediction)
print ( "Accuracy: ",  mtr.getAccuracyCount(np.array(my_prediction)))
mtr.plot_matched_counts(my_prediction.values)
#mtr.print_result(my_prediction)
mtr.print_predictions(my_prediction)


In [ ]:
[np.unique1d(a,b) for a,b in zip()]

In [ ]:
test_data = mtr.get_test_data()
#X_test = mtr.modified_dataset(getAllData(test_data)) #
X_test = test_data[X.columns]
#scaler.fit(X_test)
X_test_scaled = scaler.transform(X_test.drop('T',axis=1))
#[X.columns])
X_test_scaled = X_test
#[X.columns]
test_result = pd.DataFrame(index=X_test.index)
test_result['T'] = test_data['T']

for i in range(1,8):
    test_result['N'+str(i)+'_P'] = model_store[i-1].predict(X_test_scaled).round()
#     test_result['N'+str(i)+'_P0'] = test_result['N'+str(i)+'_P']-1
#     test_result['N'+str(i)+'_P1'] = test_result['N'+str(i)+'_P']+1


#mtr.print_result(test_result)
mtr.print_predictions(test_result.astype(int))



In [ ]:
predicted_result = np.unique(result.values)
print(result.values)
predicted_result = [np.unique(a) for a in result.values]
len(predicted_result)